#Lab.03 / IBM3202 – Análisis filogenético usando biopython y RAxML

## Aspectos teóricos

En el centro de la bioinformática y genómica llace el análisis filogenético
 y busca establecer las relaciones evolutivas entre diferentes secuencias de proteína o ADN homólogos y sus secuencias ancestrales (ancestros comunes) de las que las primeras emergieron. Un resultado típico de un análisis filogenético es el  **árbol filogenético**, como el siguiente:

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/phylo_01.png' width=500/>
<figcaption>Figura 1. Un árbol filogenético visualmente representa una hipóteiss de cómo se relaciona un grupo de secuencias. Esta figura explora cómo la conectividad en las partes del árbol entrega información. Este árbol muestra como las siete secuencias en las puntas de las ramas, llamadas taxa, están relacionadas. Cada rama horizontal representa un linage evolutivo. El largo de la rama es arbitrario a menos que el diagrama especifique que los largos de ellas entregan información como tiempo o cambio genético. Cada nodo representa el ancestro común de los linages evolutivos que difieren de él. Una de los nodos representa el ancestro común  de todas las secuencias presentes en el árbol.
<br>Urry LA et al (2020). Campbell Biology. <i>Pearson Education, 12th Ed</i></figcaption></center>
</figure>

Em esta representación, las puntas del árbol (hojas) corresponden a diferente secuencias o genes que existen en la actualidad (llamados **taxa**), por lo tanto representa información real. Los **nodos** o puntos de ramas conectando dos secuencias son puntos de divergencia en la evolución de secuencias, llamados **ancestros comunes** de los taxa extantes. La conección entre dos o más secuencias con una secuencia ancestral extinta e hipotética permite agrupar las secuencias en **clados**. Por otro lado, los largos de las ramas representan los cambios evolutivos entre un ancestro y sus descendientes. Tomar nota que estamos diciendo **'secuencias'** y no **'especies'**, ya que tal argumento require la suposición de aislamiento genético, es decir, que todas las secuencias en una especie dada muestra la misma deriva genética, pero aprendimos de las clases que existe la transferencia horizontal de genes.

Muchas cosas pueden ser dichas acerca de este árbol. Primero, un **árbol filogenético** está basado en un alineamiento múltiple de secuencia, y por lo tanto es **tan bueno como el alineamiento**. Por ejemplo, la politomía en el nodo ancestral que conecta los taxa D, E y F; la información entre estas secuencias no permite discernir cómo estas secuencias evolucionaron desde un ancestro común, es decir, imaginemos que hay dos mutacions y el análisis filogenético no sabe cuál viene primero.

Segundo, el taxon G está definido como un **taxon basal**, un linage que diverge de todos los otros miembros de su grupo temprano en la historia evolutiva.

Finalmente, el árbol filogenético está **enraizado** a través de un **grupo foráneo u *outgroup***, es decir, una secuencia o grupo de secuencias que es más distantemente relacionado al grupo interior de secuencias, de lo que el grupo interior de secuencias están relacionadas entre sí; un homólogo distante. Como un *outgroup* no es fácil de definir para todas las inferencias filogenética, un amplio número de árboles filogenéticos en la literatura no están enraizados, es decir que no tienen *outgroup*.

Un árbol filogenético es una estamción de las posibles rutas de evolución de un linaje ancestral a las secuencias actuales que depende de muchas variables. Iremos paso a paso a través de lo necesario para realizar esta inferencia, desde alineamiento de secuencia hasta la "resurrección" de la secuencia ancestral más probable.

## Generalidades

En este tutorial, trabajaremos con la familia de factores de transcripción de la caja Forkhead (Fox), una familia de proteínas que actualmente estamos estudiando [Medina E et al (2016) Biophys J 110 (11), 2349-2360; Medina E et (2020) J Mol Biol 432(19), 5411-5429] en colaboración con el [Laboratorio de Bioquímica y Biología Molecular de la Universidad de Chile](https://sites.google.com/view/labbq) .

Estas proteínas son controladoras maestras de la expresión génica en varios eucariotas desde los primeros Bilateria, y recientemente se ha determinado que también participan en la remodelación cromosómica, ya que su estructura es similar a la estructura de las histonas. En humanos, existen 19 subfamilias diferentes de Fox (de A a S), y casi todas corresponden a proteínas monoméricas, excepto FoxP. Se sabe que los miembros de la subfamilia FoxP forman dímeros para permitir la formación de interacciones cromosómicas a larga distancia debido a una mutación clave de Pro a Ala en su secuencia (ver abajo).

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/phylo_02.png'/>
<figcaption>FIGURA 2. Determinantes de secuencia de las acrobacias estructurales del dominio de unión al ADN de los factores de transcripción FoxP humanos. El panel izquierdo muestra cómo las proteínas FoxP se oligomerizan intercambiando elementos de estructura secundaria idénticos entre subunidades adyacentes, un mecanismo conocido como intercambio de dominios. Se indica un residuo clave de Ala validado experimentalmente que permite el intercambio de dominios y se ha demostrado que está conservado en todas las proteínas FoxP humanas y reemplazado por Pro en todos los miembros monoméricos de Fox, como se muestra en la alineación de secuencias múltiples a la derecha.
<br> Medina E et al (2016) <i>Biophys J 110 (11), 2349-2360</i>; Stroud JC et al (2008) <i>Structure 14(1), 159-66</i></figcaption></center>
</figure>

Hoy, inferiremos dónde surgió la novedad evolutiva de formar dímeros durante la evolución de las proteínas Fox utilizando **RAxML** (Randomized Axelerated Maximum Likelihood), un programa popular para análisis filogenéticos de grandes conjuntos de datos bajo máxima verosimilitud.


#Part 0. Descargando e instalando los programas

Antes de comenzar, primero **recuerda iniciar el entorno de ejecución alojado en Google Colab**.

Luego, debemos instalar varios programas para llevar a cabo este tutorial. En particular:
- **biopython** para buscar, recuperar, analizar y almacenar secuencias de ADN y proteínas.
- **MAFFT**, un programa de alineamiento múltiple de secuencias para sistemas operativos similares a Unix que ofrece una variedad de métodos de alineamiento múltiple.
- **miniconda**, un instalador mínimo gratuito de **conda** para la gestión de paquetes de software y entornos.
- **ModelTest-ng**, una herramienta para seleccionar el modelo de evolución más adecuado para alineamiento de ADN y proteínas.
- **RAxML-ng**, una herramienta de inferencia de árboles filogenéticos que utiliza el criterio de optimización de máxima verosimilitud (ML).

Después de realizar varias pruebas, las siguientes instrucciones de instalación son la mejor manera de configurar **Google Colab** para esta sesión de laboratorio.

1. Primero, instalaremos biopython de la siguiente manera:


In [ ]:
#Instalar biopython usando pip
!pip install biopython

In [ ]:
#Importando biopython y sistema operativo
import os
import sys
import Bio

2. Después instalamos MAFFT

In [ ]:
#instalamos mafft usando apt-get en modo silencioso
!apt-get install -qq -y mafft

In [ ]:
#revisamos que mafft haya quedado instalado
!mafft --help

3. Finalmente, instalaremos conda para poder instalar RAxML-NG y ModelTest-NG de la siguiente manera:


In [ ]:
# Instalar conda usando la nueva biblioteca conda-colab
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

# Instalar RAxML-NG y ModelTest-NG desde
# el repositorio bioconda
!conda install -c bioconda raxml-ng modeltest-ng --yes

# MAFFT también podría instalarse con conda
# usando !conda install -c bioconda -y mafft
# Optamos por apt-get en su lugar

#Part I - Obtener secuencias de FoxP usando BLAST

Aquí, presentaremos dos opciones diferentes para recuperar secuencias de proteínas Fox.

1. Recuperar secuencias de proteínas utilizando la versión web de BLAST.
2. Recuperar secuencias de proteínas utilizando BLAST a través de Biopython.

##Part I.A - Obtener las secuencias Fox Usando la versión web de BLAST

1. Comenzaremos abriendo una nueva ventana con [BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi). Aquí, selecciona **blastp** para secuencias de proteínas. Luego, en el cuadro *“Enter accession number(s), gi(s) or FASTA sequence(s)”*, escribe **2KIU_A**.

  También, indicaremos a BLAST que solo busque secuencias humanas (opción **Search Set** **Organism: humans**) en el Banco de Datos de Proteínas (opción **Search Set** **Database: Protein Data Bank (pdb)**), como se muestra a continuación.


**Pregunta:** Hay diferentes tipos de BLAST, qué hace cada uno de ellos?

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/phylo_03.png'/>
</center>
</figure>

2. Los resultados de este BLAST se muestran a continuación. Es importante destacar que el primer resultado es, de hecho, nuestra secuencia de consulta: **2KIU**. Además, todas las demás secuencias tienen códigos similares de 4 caracteres.

  Si recuerdas nuestro tutorial anterior sobre visualización de la estructura de proteínas usando **py3Dmol**, estos códigos de 4 caracteres corresponden a los identificadores de acceso del Banco de Datos de Proteínas (PDB). Estas estructuras siempre van acompañadas de una secuencia FASTA de la proteína cristalizada.

  **Te recomendamos encarecidamente** que accedas al [sitio web del PDB](https://www.rcsb.org/) y analices la información que está depositada para cada una de estas estructuras.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/phylo_04.png'/>
</center>
</figure>

3. Descargaremos algunas de estas secuencias para nuestro trabajo. Para recopilar estas secuencias para análisis futuros (por ejemplo, alineamientos de múltiples secuencias), necesitamos seleccionar las secuencias que nos gustaría descargar, hacer clic en **Download** y seleccionar la opción **FASTA (complete sequences)**. Luego, puedes cargar este archivo o copiar estas secuencias en un nuevo archivo en Google Colab.


**PREGUNTA:** ¿Cuáles son los parámetros de BLAST que serían importantes para decidir qué secuencias recopilar y cuáles descartar? ¿Podemos incluir más secuencias?


**💡 CONSEJO:** Ten en cuenta que estamos utilizando la base de datos de proteínas PDB solo con fines de este tutorial, ya que tiene un número limitado de secuencias de proteínas (debido al número limitado de estructuras). Sin embargo, es posible que encuentres más adecuado el uso de otra base de datos para tu trabajo con tus propias secuencias de proteínas o ADN, ya que contienen un mayor número de secuencias y, por lo tanto, una mayor variabilidad/redundancia de secuencias.


## Part I.B - Obtener las secuencias de las proteínas Fox usando biopython

Biopython es un excelente compañero para trabajar con secuencias de ADN y proteínas, así como con estructuras. Aquí, mostraremos cómo usarlo para recuperar secuencias de proteínas Fox.

1. Primero, comenzaremos utilizando _Entrez_ para recuperar la secuencia del código de acceso **2KIU_A** en formato FASTA y _SeqIO_ para poder leer, analizar y/o escribir esta secuencia. Además de descargar nuestra secuencia, también la almacenaremos en una cadena (aquí denominada _aaseq_) para su uso posterior en este tutorial.


In [ ]:
from Bio import SeqIO, Entrez

# Configurando tu correo electrónico para poder usar Entrez
Entrez.email = 'tu.correo@uc.cl'

# Aquí, configuramos un control temporal con nuestra secuencia descargada en formato fasta
temp = Entrez.efetch(db="protein", rettype="fasta", id="2KIU_A")

# Creando un archivo fasta para escribir nuestra secuencia descargada
aaseq_out = open("2KIU_A.fasta", 'w')

# Leyendo la información de la secuencia como una cadena en formato fasta
aaseq = SeqIO.read(temp, format="fasta")

# Escribiendo el registro de la secuencia en formato fasta
SeqIO.write(aaseq, aaseq_out, "fasta")

# Cerrando tanto el control temporal como el archivo FASTA
temp.close()
aaseq_out.close()


2. Lo grandioso de _SeqIO_ es que puedes usarlo para manipular tu secuencia (por ejemplo, ordenar, cambiar formatos, etc.) y también para imprimir información sobre tu secuencia, como su descripción, secuencia e ID de acceso.

   Puedes probar estos comandos a continuación escribiendo **"aaseq."** y luego seleccionando una de las opciones de autocompletado sugeridas por Google Colab. Con esta información, intenta obtener los detalles solicitados a continuación.


In [ ]:
# Imprimiendo el número de aminoácidos como ejemplo
print("Longitud de la secuencia (aa):")
print(len(aaseq))

In [ ]:
# Imprimiendo la descripción de la secuencia
print("La descripción de la secuencia es:")
print(aaseq.description)

# Imprimiendo el ID de la secuencia
print("El ID de la secuencia es:")
print(aaseq.id)

# Imprimiendo la secuencia
print("La secuencia es:")
print(aaseq.seq)

**💡 CONSEJO:** Si solo tienes una secuencia de proteínas/ácidos nucleicos en lugar de un ID de acceso, puedes almacenar la secuencia en una cadena utilizando la función _**Seq**_, como se muestra a continuación.

In [ ]:
# En caso de que no tengas un ID de acceso, sino solo una secuencia
from Bio.Seq import Seq
my_seq = Seq("AEVRPPFTYASLIRQAILESPEKQLTLNEIYNWFTRMFPYFRRNAATWKNAVRHNLSLHKYFVRVENVKGAVWTVDEVEFQKRRPQK")


3. Una vez que ya hemos almacenado la información de nuestra secuencia de consulta en una cadena, podemos usarla para realizar una búsqueda BLAST dentro de Google Colab a través de Biopython mediante _NCBIWWW_.

  El bloque de código a continuación especifica el programa de blast **blastp**, la base de datos **pdb** y la secuencia de consulta. Este proceso no debería tardar más de 2 minutos.

  **💡 CONSEJO:** Ten en cuenta que estamos utilizando la base de datos de proteínas PDB solo con fines de este tutorial, ya que tiene un número limitado de secuencias de proteínas (debido al número limitado de estructuras). Sin embargo, es posible que encuentres más adecuado el uso de otra base de datos para tu trabajo con tus propias secuencias de proteínas o ADN, ya que contienen un mayor número de secuencias y, por lo tanto, una mayor variabilidad/redundancia de secuencias. Para los fines de este tutorial, también estamos especificando recuperar solo secuencias pertenecientes a humanos.


In [ ]:
# Utilizando Biopython para realizar una búsqueda BLAST
%%time
from Bio.Blast import NCBIWWW
# NCBIWWW.qblast(program, database, sequence)
result_handle = NCBIWWW.qblast("blastp", "pdb", aaseq.seq, entrez_query='human[organism]')

4. Para analizar estos datos, necesitamos almacenarlos en un control para el postprocesamiento utilizando _NCBIXML_, como se muestra a continuación:


In [ ]:
# Leer los resultados en formato XML para analizar los registros del BLAST
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)
# Esto es necesario para restablecer las búsquedas y comenzar de nuevo
result_handle.close()

5. _NCBIXML_ permite la manipulación de datos de manera similar a lo que vimos anteriormente con _SeqIO_. En la siguiente celda de código, intenta obtener el **ID del hit** de BLAST, la **longitud** de la secuencia y el valor de **expect** o valor de e para cada par de alineaciones de alta puntuación (HSP), utilizando nuevamente la función de autocompletado de Google Colab.


In [ ]:
# Probando las funciones de análisis de NCBIXML
for alignment in blast_record.alignments:
  for hsp in alignment.hsps:
    # Ejemplo con el código de acceso
    print("ID del Hit:")
    print(alignment.hit_id)
    # Ahora, inténtalo tú mismo para imprimir la longitud de la secuencia

    print("Longitud de la secuencia del Hit:")
    # INSERTA TU PROPIO COMANDO A CONTINUACIÓN
    print()

    # Ahora imprime el valor de e para cada hit
    print("Valor de E:")
    # INSERTA TU PROPIO COMANDO A CONTINUACIÓN
    print()


6. Podemos utilizar todas estas funciones para imprimir un resumen similar a BLAST de los resultados e incluso para filtrar los resultados según parámetros como la identidad de la secuencia, la cobertura y/o el valor de e. **Examina cuidadosamente los comandos a continuación para lograr esto y la salida resultante.**


In [ ]:
# Aquí mostramos cómo establecer un umbral de identidad de secuencia
# ¡Se pueden emplear muchos otros umbrales!
PIDcut = 1.00
# Imprimiendo todos los parámetros de BLAST de manera similar al sitio web
for alignment in blast_record.alignments:
  for hsp in alignment.hsps:
    # Aquí, agregamos una condición para imprimir solo hits iguales o inferiores a un umbral de identidad de secuencia
    if (hsp.identities / hsp.align_length) <= PIDcut:
      print("Código de acceso:", alignment.hit_id)
      print("Longitud de la secuencia:", alignment.length)
      print("Longitud de alineación:", hsp.align_length)
      print("Valor de E:", hsp.expect)
      # El siguiente comando calcula la identidad de la secuencia en relación con
      # la longitud de la alineación
      print("Identidad de la secuencia [%]:", "{:.2f}".format(100 * hsp.identities / hsp.align_length))
      # El siguiente comando calcula la cobertura de la secuencia en relación con
      # la longitud de la secuencia
      print("Cobertura de la secuencia [%]:", "{:.2f}".format(100 * sum(c.isalpha() for c in hsp.query) / len(aaseq)))
      print()
      # Aquí imprimimos los primeros 60 caracteres de la consulta y el hit y sus coincidencias
      print("consulta:", hsp.query[0:60] + "...")
      print("      ", hsp.match[0:60] + "...")
      print("sbjct:", hsp.sbjct[0:60] + "...")
      print()


**PREGUNTA:** ¿Cuáles son los parámetros de BLAST que serían importantes para decidir qué secuencias recopilar y cuáles descartar? ¿Podemos incluir más secuencias?


**📚 TAREA:** Copia la celda de código anterior y edítala para agregar otros filtros, como un valor mínimo de E y cobertura de secuencia.


7. Después de la búsqueda y filtrado de BLAST, nos gustaría recuperar todas las secuencias que coinciden con nuestros criterios de selección. Aquí, optamos por descargar las 20 mejores secuencias, pero puedes utilizar filtros como la cobertura mínima de la secuencia, el valor mínimo de e y el PID máximo para limitar tus secuencias de salida.


In [ ]:
# Configurando tu correo electrónico para poder usar Entrez
Entrez.email = 'tu.correo@uc.cl'

# Generar un bucle para escribir todas las secuencias en un archivo de salida
with open("sequences.fasta", "a") as todas_las_coincidencias_salida:
    # Ver cómo estamos indicando usar las 20 mejores coincidencias
    for alignment in blast_record.alignments[:20]:
        for hsp in alignment.hsps:
            # Aquí, agregamos una condición para imprimir solo secuencias por debajo de un umbral de PID
            if (hsp.identities / len(hsp.match)) <= PIDcut:
                print("Recuperando la secuencia de la proteína:", alignment.hit_id)
                fetch = Entrez.efetch(db="protein", id=alignment.hit_id, rettype="fasta")
                # Leyendo la secuencia almacenada en la cadena temporal en formato fasta
                todas_las_coincidencias_seqs = SeqIO.read(fetch, format="fasta")
                # Escribiendo la secuencia y su ID en formato fasta
                SeqIO.write(todas_las_coincidencias_seqs, todas_las_coincidencias_salida, "fasta")
                fetch.close()
# Cerrar el efetch y el archivo
todas_las_coincidencias_salida.close()


😱 **COPIA DE SEGURIDAD DE EMERGENCIA:** En caso de que BLASTP falle debido a un problema con los servidores de NCBI, puedes descargar un archivo FASTA que contiene 20 homólogos de secuencias de proteínas generadas el 27 de septiembre de 2021:


In [ ]:
!wget https://github.com/pb3lab/ibm3202/raw/master/files/emergency_backup/lab03/sequences.fasta -O sequences.fasta

#Part II - Genera y edita un alineamiento múltiple de secuencia (MSA) usando MAFFT y biopython

Una vez que se resuelva BLAST, podemos proceder con el **Alineamiento Múltiple de Secuencias (MSA)**. En este caso, primero utilizaremos **MAFFT** para alinear todas las secuencias recuperadas.

1. Para realizar un alineamiento MSA utilizando MAFFT, podemos nuevamente utilizar el envoltorio de biopython _Bio.Align.Applications_, como se muestra en la celda de código a continuación.


In [ ]:
from Bio.Align.Applications import MafftCommandline
mafft_cline=MafftCommandline(input="sequences.fasta")
print(mafft_cline)
stdout, stderr = mafft_cline()
with open("aligned.fasta", "w") as handle:
  handle.write(stdout)
from Bio import AlignIO
align = AlignIO.read("aligned.fasta", "fasta")

2. El resultado de este algoritmo, que es una extensión de los **Algoritmos de Alineamiento de Pares** que discutimos durante las conferencias, se puede ver en un visor MSA en línea como [Alignment Viewer 2.0](https://fast.alignmentviewer.org/). Sin embargo, algunos desarrollos recientes utilizan widgets de **Panel** y la biblioteca de gráficos interactivos **Bokeh** para Python, diseñados para su uso en navegadores web y paneles. Aquí, estaremos utilizando uno de esos desarrollos.

In [ ]:
#@title Visor de MSA de Proteínas en Google Colab
# El siguiente código está modificado del maravilloso visor desarrollado por Damien Farrell
# https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner

# Importando todos los módulos primero
import os, io, random
import string
import numpy as np

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

import panel as pn
import panel.widgets as pnw
pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot

# Configurando el código de color de aminoácidos según el esquema de color Zappo
def get_colors(seqs):
    # Hacer colores para las bases en la secuencia
    text = [i for s in list(seqs) for i in s]
    # Utilizar el esquema de color Zappo
    clrs =  {'K':'red',
             'R':'red',
             'H':'red',
             'D':'green',
             'E':'green',
             'Q':'blue',
             'N':'blue',
             'S':'blue',
             'T':'blue',
             'A':'blue',
             'I':'blue',
             'L':'blue',
             'M':'blue',
             'V':'blue',
             'F':'orange',
             'Y':'orange',
             'W':'orange',
             'C':'blue',
             'P':'yellow',
             'G':'orange',
             '-':'white'}
    colors = [clrs[i] for i in text]
    return colors

# Configurando el visor de MSA
def view_alignment(aln, fontsize="9pt", plot_width=800):
    """Vista de alineación de secuencias Bokeh"""

    # Hacer listas de secuencias e identificadores desde el objeto aln
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)
    N = len(seqs[0])
    S = len(seqs)
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    # Crea una rejilla 2D de coordenadas a partir de los arrays 1D
    xx, yy = np.meshgrid(x, y)
    # Aplana los arrays
    gx = xx.ravel()
    gy = yy.flatten()
    # Usa recty para las coordenadas rect con un desplazamiento
    recty = gy+.5
    h= 1/S
    # Ahora podemos crear el ColumnDataSource con todos los arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')
    if N>100:
        viewlen=100
    else:
        viewlen=N
    # view_range es para la vista cercana
    view_range = (0,viewlen)
    tools="xpan, xwheel_zoom, reset, save"

    # vista de la secuencia completa (sin texto, con zoom)
    p = figure(title=None, width= plot_width, height=50,
               x_range=x_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False

    # vista de texto de la secuencia con capacidad para desplazarse a lo largo del eje x
    p1 = figure(title=None, width=plot_width, height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0

    p = gridplot([[p],[p1]], toolbar_location='below')
    return p

# Cargando el visor indicando el archivo y formato de MSA a leer
#@markdown Nombre del archivo MSA (incluyendo el tipo de archivo)
MSAfile = 'aligned.fasta' #@param {type:"string"}
MSAformat = 'fasta' #@param {type:"string"}
aln = AlignIO.read(MSAfile, MSAformat)
p = view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p)


3. Como puedes ver, las secuencias están alineadas, ya que muchos residuos conservados entre diferentes secuencias ocupan las mismas columnas dentro del alineamiento. Sin embargo, **también podemos ver algunas secuencias que son más largas en los extremos de la proteína**. Estos extremos no contribuirán nada al alineamiento, ya que no hay nada con qué compararlos. Por lo tanto, recortaremos las secuencias en ambos extremos N y C seleccionando las regiones que queremos eliminar.

  Idealmente, deberías corregir manualmente el alineamiento y recortar los extremos de las secuencias, y **te animamos a hacerlo**. Debido a restricciones de tiempo, incluimos un script a continuación que recorta los extremos N y C del alineamiento en función de encontrar las primeras y últimas columnas del MSA sin ningún espacio.


In [ ]:
import sys
from Bio import AlignIO
aln = AlignIO.read("aligned.fasta", "fasta")

for fcol in range(aln.get_alignment_length()):
  if not "-" in aln[:, fcol]:
    position1 = fcol
    print("First full column is {}".format(fcol))
    break
for lcol in reversed(range(aln.get_alignment_length())):
  if not "-" in aln[:, lcol]:
    position2 = lcol+1
    print("Last full column is {}".format(lcol))
    break

print("New alignment:")
print(aln[:, position1:position2])

with open("aligned_trimmed.fasta", "w") as handle:
  count = (SeqIO.write(aln[:, position1:position2], handle, "fasta"))

trim = AlignIO.read("aligned_trimmed.fasta", "fasta")

4. También necesitamos **asegurarnos de que no haya secuencias duplicadas incluidas en él**. Tener información redundante no aporta ningún beneficio para el análisis filogenético que estamos llevando a cabo aquí.

¿Tenemos secuencias duplicadas? Podemos generar rápidamente una matriz de distancias basada en las diferencias entre pares de todas las secuencias en el archivo de alineamiento utilizando la herramienta _DistanceCalculator_ de Biopython. Por lo tanto, un par de secuencias con 0.0 diferencias corresponderían a secuencias equivalentes (identidad de secuencia del 100%).


In [ ]:
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio import AlignIO
aln = AlignIO.read(open('aligned_trimmed.fasta'), 'fasta')
calculator = DistanceCalculator('identity')
dm = calculator.get_distance(aln)
print(dm)

5. Hay programas adecuados para eliminar las secuencias duplicadas que detectaste en el ejercicio anterior, como **CD-HIT**, pero aquí optamos por ejecutar directamente otro script que compara las secuencias en el alineamiento y elimina las duplicadas.


In [ ]:
# Secuencia del script limpiador
# Modificado desde https://peterjc.github.io/wiki/Sequence_Cleaner
from Bio import SeqIO

def sequence_cleaner(fasta_file, min_length=0):
  # Creamos nuestra tabla hash para agregar las secuencias
  sequences = {}
  # Utilizando el análisis de fasta de Biopython, podemos leer nuestra entrada fasta
  for seq_record in SeqIO.parse(fasta_file, "fasta"):
    # Tomamos la secuencia actual
    sequence = str(seq_record.seq).upper()
    # Verificamos si la secuencia actual cumple con los parámetros del usuario
    if (len(sequence) >= min_length):
      # Si la secuencia pasa la prueba "¿está limpia?" y no está en la
      # tabla hash, la secuencia y su identificador estarán en la tabla hash
        if sequence not in sequences:
          sequences[sequence] = seq_record.id
      # Si ya está en la tabla hash, simplemente vamos a concatenar el ID
      # de la secuencia actual a otra que ya esté en la tabla hash
        else:
          sequences[sequence] += "_" + seq_record.id

  # Escribimos las secuencias limpias
  # Creamos un archivo en el mismo directorio donde se ejecutó este script
  with open("clear_" + fasta_file, "w+") as output_file:
  # Simplemente leemos la tabla hash y escribimos en el archivo en formato fasta
    for sequence in sequences:
      output_file.write(">" + sequences[sequence] + "\n" + sequence + "\n")
  print("¡LIMPIO!\nPor favor, verifica clear_" + fasta_file)


In [ ]:
sequence_cleaner('aligned_trimmed.fasta', 0)

**PREGUNTA:** ¿Qué sucedería con nuestro análisis filogenético y el árbol filogenético si no eliminamos estas secuencias duplicadas?

6. Nuestro MSA alineado, recortado y curado se ve como se muestra a continuación:


In [ ]:
# Visor de MSA de proteínas en Google Colab
# El siguiente código está modificado del maravilloso visor desarrollado por Damien Farrell
# https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner

# Importando todos los módulos primero
import os, io, random
import string
import numpy as np

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

import panel as pn
import panel.widgets as pnw
pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot

# Configurando el código de colores de aminoácidos según el esquema de colores de Zappo
def get_colors(seqs):
    # Hacer colores para bases en la secuencia
    text = [i for s in list(seqs) for i in s]
    # Utilizar el esquema de colores de Zappo
    clrs =  {'K':'red',
             'R':'red',
             'H':'red',
             'D':'green',
             'E':'green',
             'Q':'blue',
             'N':'blue',
             'S':'blue',
             'T':'blue',
             'A':'blue',
             'I':'blue',
             'L':'blue',
             'M':'blue',
             'V':'blue',
             'F':'orange',
             'Y':'orange',
             'W':'orange',
             'C':'blue',
             'P':'yellow',
             'G':'orange',
             '-':'white'}
    colors = [clrs[i] for i in text]
    return colors

# Configurando el visor de MSA
def view_alignment(aln, fontsize="9pt", plot_width=800):
    """Vista de alineación de secuencias Bokeh"""

    # Hacer listas de secuencias e IDs a partir del objeto de alineación
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)
    N = len(seqs[0])
    S = len(seqs)
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    # Crear una cuadrícula 2D de coordenadas a partir de las matrices 1D
    xx, yy = np.meshgrid(x, y)
    # Aplanar las matrices
    gx = xx.ravel()
    gy = yy.flatten()
    # Usar recty para las coordenadas del rectángulo con un desplazamiento
    recty = gy+.5
    h= 1/S
    # Ahora podemos crear el ColumnDataSource con todas las matrices
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')
    if N>100:
        viewlen=100
    else:
        viewlen=N
    # view_range es para la vista de primer plano
    view_range = (0,viewlen)
    tools="xpan, xwheel_zoom, reset, save"

    # vista de secuencia completa (sin texto, con zoom)
    p = figure(title=None, width= plot_width, height=50,
               x_range=x_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False

    # vista de texto de secuencia con capacidad para desplazarse a lo largo del eje x
    p1 = figure(title=None, width=plot_width, height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0

    p = gridplot([[p],[p1]], toolbar_location='below')
    return p

# Cargando el visor indicando el archivo de MSA y el formato a leer
#@markdown Nombre del archivo de MSA (incluyendo el tipo de archivo)
MSAfile = 'clear_aligned_trimmed.fasta' #@param {type:"string"}
MSAformat = 'fasta' #@param {type:"string"}
aln = AlignIO.read(MSAfile,MSAformat)
p = view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p)

#Part III - Inferencia filogenética y reconstrucción de secuencias ancestrales usando RAxML

Utilizaremos nuestra alineación final, recortada y filtrada para inferir las relaciones filogenéticas de nuestras secuencias con el método de búsqueda de árboles de **RAxML** utilizando el criterio de optimalidad de **Máxima Verosimilitud** (ML) de manera **heurística**.

Aunque ya discutimos algunos de los métodos filogenéticos, así como los modelos evolutivos durante nuestra Conferencia, **te invitamos nuevamente a leer el Apéndice de la Conferencia sobre ML**.

Lo que no discutimos fue que los modelos evolutivos también podrían incorporar parámetros adicionales, como sitios invariables (conservación) o variaciones en las tasas de sustitución entre sitios. Por ejemplo, los **codones de iniciación (típicamente, ATG)** pueden no ser libres de variar en absoluto. Por lo tanto, deberíamos hacer que esos sitios sean invariantes. Por otro lado, sabemos que las posiciones en el interior de una proteína evolucionan más lentamente que los residuos en la superficie, por lo que las sustituciones tienen **tasas variables** dependiendo de la posición de estos residuos. En el caso de la variación, modelar estas tasas variables consume recursos computacionales, por lo que se utiliza una distribución **gamma** bien comportada y matemáticamente tratable para modelar estas variaciones en las tasas.

1. Ahora comenzaremos a realizar algunas filogenias de Máxima Verosimilitud (ML), pero **¿qué modelo deberíamos usar?** Bueno, ML tiene la ventaja de que podemos utilizar la **verosimilitud** para inferir qué modelo evolutivo es más adecuado para la alineación que estamos utilizando (¿interesante, verdad?). Esto está implementado en el programa **ModelTest-NG** y se puede usar de la siguiente manera:


In [ ]:
!modeltest-ng -i clear_aligned_trimmed.fasta -d aa

El resultado se verá muy complejo e ilegible. Sin embargo, los modelos se seleccionan según **AICc**, que determina la idoneidad de un modelo dado si minimiza mejor **-lnL** con el menor número de parámetros. Aquí, **lnL** es la verosimilitud logarítmica, y su valor negativo se utiliza como objetivo de minimización durante la selección del modelo. **En nuestro caso particular, el mejor modelo resultó ser LG+G4**, o el modelo de **Le-Gascuel** con variación **Gamma**.

2. Ahora, podemos usar nuestra MSA final y este modelo evolutivo para realizar nuestro primer análisis filogenético utilizando **RAxML-NG**.


In [ ]:
!raxml-ng --msa clear_aligned_trimmed.fasta --model LG+G4 --prefix T1 --threads 2

3. Una vez más, podemos utilizar Biopython para ver los resultados de nuestra inferencia filogenética mediante la herramienta _Phylo_.


In [ ]:
from Bio import Phylo
tree = Phylo.read("T1.raxml.bestTree", "newick")
Phylo.draw(tree)

**Pregunta:** ¿Cuántos clados ves? ¿Cuántos ancestros comunes hay? ¿Quiénes son los miembros del clado que contiene nuestra secuencia de consulta?

4. Ahora necesitamos probar la confiabilidad (reproducibilidad) de nuestro árbol filogenético utilizando el **método de boostrapping** que discutimos durante nuestras conferencias. Recuerda que esto no es parte del método de construcción del árbol, sino una prueba filogenética.

  Aunque generalmente se sugieren de 1000 a 2000 réplicas de bootstrap para determinar la confianza del árbol filogenético, RAxML también tiene un método llamado **bootstopping** que determina cuántas réplicas de bootstrap se requieren para obtener valores de soporte estables. Sin embargo, para un tutorial rápido, hemos solicitado solo 100 réplicas de bootstrap.


In [ ]:
!raxml-ng --bootstrap --msa T1.raxml.rba --model LG+G4 --prefix T2 --threads 2 --bs-tree 100

**PREGUNTA:** Un clado confiable debería tener valores de boostrapping **> 70**. ¿Qué significaría un valor de boostrapping de 70?

**💡 CONSEJO:** Aún puedes verificar la convergencia de la prueba de boostrapping después ejecutando el siguiente comando:

`!raxml-ng --bsconverge --bs-trees T2.raxml.bootstraps --prefix Test --threads 2 --bs-cutoff 0.03`

En la práctica, un valor de corte de convergencia del 3% debería ser suficiente en la mayoría de los casos.




5. Ahora, mapearemos los valores de soporte obtenidos de la prueba de boostrapping en el árbol ML mejor calificado en la MSA original. Una vez que hayas ejecutado la celda de código a continuación, utiliza nuevamente el paquete Phylo para mostrar el árbol filogenético con sus valores de boostrapping.


In [ ]:
!raxml-ng --support --tree T1.raxml.bestTree --bs-trees T2.raxml.bootstraps --prefix T3 --threads 2

In [ ]:
#Aquí, ¡usa nuevamente el paquete Phylo de Biopython!
#AGREGA UN CÓDIGO A CONTINUACIÓN PARA DIBUJAR ESTE ÁRBOL CON LOS VALORES DE BOOTSTRAP
from Bio import Phylo
tree = Phylo.read("T3.raxml.support", "newick")
Phylo.draw(tree)


6. Con esto, hemos terminado de inferir nuestros árboles filogenéticos. También podemos obtener y descargar el archivo **YourFileName.raxml.bestTree** (que es el archivo del árbol) y verlo en el sitio de visualización [**iTOL**](https://itol.embl.de/).

En la parte superior de este sitio, hay un botón **UPLOAD**. Podemos presionar el botón y cargar el archivo besttree para dibujar nuestro árbol filogenético de manera clara.


7. Ahora usaremos el árbol de ML para obtener una estimación de la **secuencia ancestral** para la clade de FoxP. El ADN que codifica estas secuencias de proteínas suele ser sintetizado por los investigadores para evaluar su estructura y función, lo que permite inferir los contextos celulares, ambientales y funcionales de los organismos ancestrales y existentes.

Dado que ya tenemos la MSA y el árbol de ML, no necesitamos nada más para obtener las secuencias ancestrales en cada uno de los nodos internos. Simplemente usa el código a continuación.



In [ ]:
!raxml-ng --ancestral --msa clear_aligned_trimmed.fasta --tree T1.raxml.bestTree --model LG+G4 --prefix ASR

8. Ahora, abre el archivo **YourFileName.raxml.ancestralStates** para ver las secuencias más probables basadas en las probabilidades para cada posición de la alineación (que están contenidas en el archivo **YourFileName.raxml.ancestralProbs**).

Pero, ¿qué secuencia corresponde a qué nodo? Ahora puedes dibujar **YourFileName.raxml.ancestralTree** para revisar los números de cada nodo y determinar qué secuencia corresponde a la clade de FoxP.


In [ ]:
#aquí usamos el paquete filo de bio de nuevo!
from Bio import Phylo
tree = Phylo.read("ASR.raxml.ancestralTree", "newick")
Phylo.draw(tree)

**📚 TAREA:** Ahora, como ejercicio final, compara la secuencia del nodo ancestral de la clade FoxP con las secuencias mostradas en la Visión general para FoxP y para los homólogos de FoxP. Además, echa un vistazo a las estructuras de los diferentes homólogos de Fox que tienes en tu alineación buscando los códigos de acceso en el [Protein Data Bank](https://www.rcsb.org).

Basándote en la información proporcionada en la Visión general de este tutorial, el análisis filogenético de las proteínas FoxP y sus homólogos, las secuencias ancestrales reconstruidas para cada nodo y tus observaciones de las diferentes estructuras incluidas en tu alineación de secuencias:

1. ¿Podemos decir algo sobre el estado oligomérico más probable de esta proteína basándonos en esta secuencia?

2. ¿Podemos decir algo sobre cómo esta subfamilia adquirió la capacidad de formar dímeros durante la evolución de las proteínas Fox?

3. ¿Hay algún elemento que no hayamos tenido en cuenta que podría alterar significativamente los resultados de nuestro análisis?

**Esto concluye el tercer tutorial. ¡Buena ciencia!**
